In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import re
import os
import matplotlib.pyplot as plt
import nltk
from collections import OrderedDict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to /home/riya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/riya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/riya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
relative_path_sdg = "../../sdg_data.csv"
relative_path_attr = "../Dictionary/Attributes_2019-20.csv"
relative_string_map = "./Data_Target/String_map.csv"
relative_semantic = "./Data_Target/Semantic_map.csv"
relative_sem_threshold = "./Data_Target/Semantic_threshold_map.csv"
relative_rank = "./Data_Target/Ranking.csv"
relative_rank_unroll = "./Data_Target/Ranking_unroll.csv"
relative_tab_filter = "./Data_Target/Ranking_tab_filter.csv"
relative_chap_filter = "./Data_Target/Ranking_chap_filter.csv"
relative_final_map = "./Mapping_Target_2019-20.csv"
relative_count = "./Data_Target/Comparison_count.csv"


# Get the absolute path by joining the current directory with the relative path
absolute_path_sdg = os.path.normpath(os.path.join(os.getcwd(), relative_path_sdg))
absolute_path_attr = os.path.normpath(os.path.join(os.getcwd(), relative_path_attr))
absolute_path_string = os.path.normpath(os.path.join(os.getcwd(), relative_string_map))
absolute_path_semantic = os.path.normpath(os.path.join(os.getcwd(), relative_semantic))
absolute_path_sem_threshold =os.path.normpath( os.path.join(os.getcwd(), relative_sem_threshold))
absolute_path_rank =os.path.normpath( os.path.join(os.getcwd(), relative_rank))
absolute_path_rank_unroll =os.path.normpath( os.path.join(os.getcwd(), relative_rank_unroll))
absolute_path_tab_filter =os.path.normpath( os.path.join(os.getcwd(), relative_tab_filter))
absolute_path_chap_filter =os.path.normpath( os.path.join(os.getcwd(), relative_chap_filter))
absolute_path_final_map =os.path.normpath( os.path.join(os.getcwd(), relative_final_map))
absolute_path_count =os.path.normpath( os.path.join(os.getcwd(), relative_count))

In [3]:
def process_result(result_sdg, sdg_df, chapter_df):
    rows = []
    
    for _, row in result_sdg.iterrows():
        attr_id = row['Rank']
        goal_no = row['Goal No.']
        target_id = row['Target_id']
        
        for attr in attr_id:
            rows.append({
                'Attr_id': attr,
                'top_n_goal_id': goal_no,
                'top_n_target_id': target_id
            })

    all_goals = pd.DataFrame(rows)
    
    goal_name = dict(zip(sdg_df['Goal No.'], sdg_df['Goal']))
    target_name = dict(zip(sdg_df['Target_id'], sdg_df['Targets']))
    
    all_goals['Goal'] = all_goals['top_n_goal_id'].map(goal_name.get)
    all_goals['Targets'] = all_goals['top_n_target_id'].map(target_name.get)
    chapter_id = dict(zip(chapter_df['Attr_id'], chapter_df['Chapter_id']))
    chapter_name = dict(zip(chapter_df['Chapter_id'], chapter_df['Chapter_name']))
    table_id = dict(zip(chapter_df['Attr_id'], chapter_df['Table_id']))
    table_name = dict(zip(chapter_df['Table_id'], chapter_df['Table_name']))
    description = dict(zip(chapter_df['Attr_id'], chapter_df['Description']))
    all_goals['Chapter_id'] = all_goals['Attr_id'].map(chapter_id.get)
    all_goals['Chapter_name'] = all_goals['Chapter_id'].map(chapter_name.get)
    all_goals['Table_id'] = all_goals['Attr_id'].map(table_id.get)
    all_goals['Table_name'] = all_goals['Table_id'].map(table_name.get)
    all_goals['Description'] = all_goals['Attr_id'].map(description.get) 
    
                
    all_goals = all_goals.loc[:,['Attr_id','Chapter_id', 'Chapter_name', 'Table_id', 'Table_name', 'Description', 'Goal', 'Targets', 'top_n_goal_id', 'top_n_target_id']]

    return all_goals


In [4]:
# Function to combine and remove duplicates from lists while maintaining order
def combine_lists_BC(row):
    list_B = row['top_n_attr_B']
    list_C = row['top_n_attr_C']
    
    # Combine lists and remove duplicates while maintaining order
    items = list(OrderedDict.fromkeys(list_B + list_C))
    return items

In [5]:
# Function to find intersection while maintaining order
def find_intersection_ABC(row):
    intersection = [x for x in row['top_n_attr_A'] if x in row['B_plus_C']]
    return intersection

In [6]:
def find_difference_A(row):
    top_n_attr = row['top_n_attr_A']
    intersection = row['A_and_BC']
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [7]:
# Function to find intersection while maintaining order
def find_intersection_BC(row):
    intersection = [x for x in row['top_n_attr_B'] if x in row['top_n_attr_C']]
    return intersection

In [8]:
def find_difference_B(row):
    top_n_attr = row['top_n_attr_B']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [9]:
def find_difference_C(row):
    top_n_attr = row['top_n_attr_C']
    intersection = set(row['A'] + row['B_and_C'] + row['A_and_BC'] + row['B_minus_all'])
    
    # Remove elements in BC and intersection from top_n_attr while preserving order
    difference = [x for x in top_n_attr if x not in intersection]
    
    return difference

In [10]:
def final_rank(row):
    #combined_top_n_attr = sorted(set(list_B + list_C), key=lambda x: (list_B + list_C).index(x))
    items = row['A_and_BC'] + row['A'] + row['B_and_C'] + row['B_minus_all'] + row['C_minus_all']
    item_set = list(OrderedDict.fromkeys(items))    
    return item_set

In [11]:
sdg = pd.read_csv(absolute_path_sdg, sep=",", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str})

In [12]:
sdg

,Goal No.,Goal,Nodal Department,Targets,Other Related Major Departments,Tentative Indicators,Target_id,Indicator_id
0,1,End poverty in all its forms everywhere,Rural Development,"By 2030, eradicate extreme poverty for all peo...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population below the interna...,1.1,1.1.1
1,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living below the ...,1.2,1.2.1
2,1,End poverty in all its forms everywhere,Rural Development,"By 2030, reduce at least by half the proportio...","Urban Development, Agriculture, Horticulture, ...","Proportion of men, women and children of all a...",1.2,1.2.2
3,1,End poverty in all its forms everywhere,Rural Development,Implement nationally appropriate social protec...,"Urban Development, Agriculture, Horticulture, ...",Percentage of the population covered by social...,1.3,1.3.1
4,1,End poverty in all its forms everywhere,Rural Development,"By 2030, ensure that all men and women, in par...","Urban Development, Agriculture, Horticulture, ...",Proportion of the population living in househo...,1.4,1.4.1
...,...,...,...,...,...,...,...,...
174,16,Strengthen the means of implementation and rev...,Finance,"Promote the development, transfer, disseminati...",Environment and Scientific Technology.,Total amount of approved funding to promote th...,16.4,16.4.1
175,16,Strengthen the means of implementation and rev...,Finance,Fully operationalize the technology bank and s...,"Environment and Scientific Technology, Inform...",Proportion of individuals using the Internet.,16.5,16.5.1
176,17,"Data, monitoring and accountability",Finance,"By 2020, enhance capacity-building support to ...","Planning, Finance, Economic and Statistics.",Proportion of sustainable development indicato...,17.1,17.1.1
177,17,"Data, monitoring and accountability",Finance,"By 2030, build on existing initiatives to deve...","Planning, Finance, Economic and Statistics.",Dollar value of all resources made available t...,17.2,17.2.1


In [13]:
sdg.drop(['Nodal Department','Other Related Major Departments', 'Tentative Indicators', 'Indicator_id'],axis=1,inplace=True)

In [14]:
# Remove duplicate rows
sdg = sdg.drop_duplicates()

In [15]:
sdg.reset_index(inplace = True, drop = True)

In [16]:
data = pd.read_csv(absolute_path_attr, sep=";", dtype={'Attr_id': str, 'Chapter_id': str, 'Table_id':str})

In [17]:
data_new = data.copy()
data_new

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description
0,3,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Taluks
1,4,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Hoblies
2,5,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Grama Panchayaths
3,6,1,General Information,1.1,Taluks Hoblies Grama Panchayath Village Accoun...,Circles
4,7,1,General Information,1.2,Inhabited Un-inhabited and Total Villages per ...,"Un-Inhabited Inhabited,Villages"
...,...,...,...,...,...,...
1548,1558,17,Other Information,17.3,Registration of Births and Deaths,"Total,Births,Late Registered events 2019"
1549,1559,17,Other Information,17.3,Registration of Births and Deaths,"Male,Deaths,Late Registered events 2019"
1550,1560,17,Other Information,17.3,Registration of Births and Deaths,"Female,Deaths,Late Registered events 2019"
1551,1561,17,Other Information,17.3,Registration of Births and Deaths,"Transgender,Deaths,Late Registered events 2019"


In [18]:
sdg_string = pd.read_csv(absolute_path_string, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_string

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,top_n_count,top_n_attr,top_n_table_id,top_n_table_name,top_n_chapter_id,top_n_chapter_name,top_n_similarities,top_n_description
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,254,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, ...",['Ration Shops and Ration Card (Below poverty ...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['General Information', 'General Information',...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Urban,Ration shops, Ration Shops and Ration ..."
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,350,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[17.2, 10.8, 10.8, 10.8, 10.8, 10.8, 10.8, 10....",['Number of Pensioners under Social Security S...,"[17, 10, 10, 10, 10, 10, 10, 10, 13, 13, 13, 1...","['Other Information', 'Education', 'Education'...","[3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...",['Bank Post Office Account (Below poverty line...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,142,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[4.3, 4.7, 11.2, 11.2, 11.2, 11.2, 12.1, 12.1,...",['Area Under Principal crops in Hectares 2018-...,"[4, 4, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,...","['Agriculture, Horticulture & Sericulture', 'A...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","['Total Pulses including Tur Bengal Gram,Pulse..."
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,363,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[11.2, 11.2, 11.5, 11.5, 11.5, 11.5, 11.5, 11....",['No.of Hospitals Doctors Beds on 31-3-2020 Cu...,"[11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1...","['Health & Family Welfare Services', 'Health &...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","['Nos.,Pvt. Hospitals Including Nursing Homes ..."
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,16,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Registration of Births and Deaths', 'Registr...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...","['Other Information', 'Other Information', 'Ot...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","['Male,Births,Registered events 2019, Registra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', 'science'...",enhance cooperation on and access to science t...,11,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[4.3, 4.7, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, 9.3, ...",['Area Under Principal crops in Hectares 2018-...,"[4, 4, 9, 9, 9, 9, 9, 9, 9, 11, 11]","['Agriculture, Horticulture & Sericulture', 'A...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","['Total Pulses including Tur Bengal Gram,Pulse..."
127,16,Strengthen the means of implementation and rev...,"Promote the development, transfer, disseminati...",16.4,"['promote', 'development', 'transfer', 'dissem...",promote the development transfer dissemination...,76,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[4.18, 4.18, 4.18, 4.18, 4.18, 4.18, 4.18, 4.1...","['Sericulture 2019-20', 'Sericulture 201

In [19]:
sdg_sim = pd.read_csv(absolute_path_semantic, sep=";", dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_sim

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,min_values,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,"[['poverty', 'people'], ['measured', 'people']...",[[-0.27236 0.229701 0.238934 ... 0.3...,"[308, 309, 311, 814, 816, 1382, 1383, 1384, 13...","[0.8575334699106769, 0.536608549286644, 0.6295...",50,"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[3, 3, 3, 5, 5, 14, 14, 14, 14, 3, 1, 1, 1, 1,...","[3.1, 3.1, 3.1, 5.4, 5.4, 14.3, 14.3, 14.3, 14...","['Working,Raingauge Stations, No. of Raingauge...","['Rainfall', 'Rainfall', 'Rainfall', 'Fisherie...",['No. of Raingauge Stations Annual Normal Rain...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,"[['reduce', 'least'], ['according', 'national'...",[[-0.070615 0.367065 0.00489179 ... -0.2...,"[996, 1366, 1367, 1379, 1380, 1381, 1382, 1383...","[0.5986987445514103, 0.8887401699967143, 0.521...",59,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[9, 14, 14, 14, 14, 14, 14, 14, 14, 14, 13, 13...","[9.2, 14.1, 14.1, 14.3, 14.3, 14.3, 14.3, 14.3...","['National Highway, P.w.d Road Length on 31-03...","['Transport and Communication', 'Rural Develop...",['P.w.d Road Length on 31-03-2020 In Kms. Cumu...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,"[['appropriate', 'social'], ['protection', 'sy...",[[-0.60930499 -0.269645 -0.01956 ... 0.1...,"[379, 465, 1160, 1161, 1214, 1215, 1216, 1217,...","[0.8510072365400712, 0.5952206115692615, 0.623...",59,"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[4, 4, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12,...","[4.3, 4.7, 11.2, 11.2, 12.1, 12.1, 12.1, 12.1,...","['Total Pulses including Tur Bengal Gram,Pulse...","['Agriculture, Horticulture & Sericulture', 'A...",['Area Under Principal crops in Hectares 2018-...
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,"[['well'], ['particular', 'poor'], ['financial...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[352, 353, 354, 1393, 1394, 1414, 574, 575, 57...","[0.7045711183911056, 0.5662583352363878, 0.658...",43,"[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[4, 4, 4, 14, 14, 15, 4, 4, 4, 4, 6, 6, 6, 6, ...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 4.18, 4.18, ...","['Nos,Tube wells,Gross Net Area Irrigated Hect...","['Agriculture, Horticulture & Sericulture', 'A...",['Gross and Net area Irrigated under different...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,"[['extreme', 'events'], ['poor', 'vulnerable']...",[[ 0.072906 0.157795 0.27599651 ... 0.7...,"[1537, 1538, 1539, 1540, 1541, 1542, 1543, 154...","[0.6873756433137852, 0.527061401761758, 0.5327...",60,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 5, 5,...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Male,Births,Registered events 2019, Registra...","['Other Information', 'Other Information', 'Ot...","['Registration of Births and Deaths', 'Registr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', 'sc

In [20]:
sdg_threshold = pd.read_csv(absolute_path_sem_threshold, dtype={'Goal No.': str, 'Target_id': str, 'Indicator_id':str}, sep=";", converters={'top_n_attr': pd.eval, 'top_n_table_id': pd.eval, 'top_n_chapter_id': pd.eval,'top_n_similarities': pd.eval})
sdg_threshold

,Goal No.,Goal,Targets,Target_id,new_targets,new_targets_str,keywords,vec,top_n_index,top_n_count,top_n_attr,top_n_chapter_id,top_n_table_id,top_n_description,top_n_chapter_name,top_n_table_name
0,1,End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1.1,"['eradicate', 'extreme', 'poverty', 'people', ...",by eradicate extreme poverty for all people ev...,"[['poverty', 'people'], ['measured', 'people']...",[[-0.27236 0.229701 0.238934 ... 0.3...,"[308, 309, 311, 814, 816, 1382, 1383, 1384, 13...",557,"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[3, 3, 3, 5, 5, 14, 14, 14, 14, 3, 3, 1, 1, 1,...","[3.1, 3.1, 3.1, 5.4, 5.4, 14.3, 14.3, 14.3, 14...","['Working,Raingauge Stations, No. of Raingauge...","['Rainfall', 'Rainfall', 'Rainfall', 'Fisherie...",['No. of Raingauge Stations Annual Normal Rain...
1,1,End poverty in all its forms everywhere,"By 2030, reduce at least by half the proportio...",1.2,"['reduce', 'least', 'half', 'men', 'woman', 'c...",by reduce at least by half the proportion of m...,"[['reduce', 'least'], ['according', 'national'...",[[-0.070615 0.367065 0.00489179 ... -0.2...,"[996, 1366, 1367, 1379, 1380, 1381, 1382, 1383...",436,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[9, 14, 14, 14, 14, 14, 14, 14, 14, 14, 17, 17...","[9.2, 14.1, 14.1, 14.3, 14.3, 14.3, 14.3, 14.3...","['National Highway, P.w.d Road Length on 31-03...","['Transport and Communication', 'Rural Develop...",['P.w.d Road Length on 31-03-2020 In Kms. Cumu...
2,1,End poverty in all its forms everywhere,Implement nationally appropriate social protec...,1.3,"['implement', 'nationally', 'appropriate', 'so...",implement nationally appropriate social protec...,"[['appropriate', 'social'], ['protection', 'sy...",[[-0.60930499 -0.269645 -0.01956 ... 0.1...,"[379, 465, 1160, 1161, 1214, 1215, 1216, 1217,...",287,"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[4, 4, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12,...","[4.3, 4.7, 11.2, 11.2, 12.1, 12.1, 12.1, 12.1,...","['Total Pulses including Tur Bengal Gram,Pulse...","['Agriculture, Horticulture & Sericulture', 'A...",['Area Under Principal crops in Hectares 2018-...
3,1,End poverty in all its forms everywhere,"By 2030, ensure that all men and women, in par...",1.4,"['ensure', 'men', 'woman', 'particular', 'poor...",by ensure that all men and women in particular...,"[['well'], ['particular', 'poor'], ['financial...",[[-0.13508999 0.35907 0.1453 ... -0.4...,"[352, 353, 354, 1393, 1394, 1414, 574, 575, 57...",421,"[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[4, 4, 4, 14, 14, 15, 4, 4, 4, 4, 6, 6, 6, 6, ...","[4.2, 4.2, 4.2, 14.5, 14.5, 15.3, 4.18, 4.18, ...","['Nos,Tube wells,Gross Net Area Irrigated Hect...","['Agriculture, Horticulture & Sericulture', 'A...",['Gross and Net area Irrigated under different...
4,1,End poverty in all its forms everywhere,"By 2030, build the resilience of the poor and ...",1.5,"['build', 'resilience', 'poor', 'vulnerable', ...",by build the resilience of the poor and those ...,"[['extreme', 'events'], ['poor', 'vulnerable']...",[[ 0.072906 0.157795 0.27599651 ... 0.7...,"[1537, 1538, 1539, 1540, 1541, 1542, 1543, 154...",502,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 1...","[17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17.3, 17....","['Male,Births,Registered events 2019, Registra...","['Other Information', 'Other Information', 'Ot...","['Registration of Births and Deaths', 'Registr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,Strengthen the means of implementation and rev...,"Enhance cooperation on and access to science, ...",16.3,"['enhance', 'cooperation', 'access', 'science'...",enhance cooperation on and access to science t...,"[['global', 'technology'], ['among', 'existing...",[[ 0.023705 -0.0712105 0.05521001 ... -0.0...,"[379, 465, 1160, 1161, 1001, 1002, 1003, 1004,...",278,"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[4

In [21]:
result_sdg_new =sdg_string.copy()
result_sdg_sim_new =sdg_sim.copy()
result_sdg_threshold_new = sdg_threshold.copy()

In [22]:
result_sdg_new = result_sdg_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [23]:
result_sdg_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."


In [24]:
result_sdg_sim_new = result_sdg_sim_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [25]:
result_sdg_sim_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13..."
1,1,1.2,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."
2,1,1.3,"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,..."
3,1,1.4,"[362, 363, 364, 1403, 1404, 1424, 584, 585, 58..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."


In [26]:
result_sdg_threshold_new = result_sdg_threshold_new[['Goal No.', 'Target_id', 'top_n_attr']]

In [27]:
result_sdg_threshold_new.head()

,Goal No.,Target_id,top_n_attr
0,1,1.1,"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13..."
1,1,1.2,"[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."
2,1,1.3,"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,..."
3,1,1.4,"[362, 363, 364, 1403, 1404, 1424, 584, 585, 58..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."


In [28]:
result_sdg_new.rename(columns = {'top_n_attr':'top_n_attr_A'}, inplace=True)

In [29]:
result_sdg_new['top_n_attr_B'] = result_sdg_threshold_new.loc[:,'top_n_attr']
result_sdg_new['top_n_attr_C'] = result_sdg_sim_new.loc[:,'top_n_attr']

In [30]:
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."
...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."


In [31]:
# Apply the function to the merged dataframe
result_sdg_new['B_plus_C'] = result_sdg_new.apply(combine_lists_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."
...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."


In [32]:
# Apply the function to the merged dataframe
result_sdg_new['A_and_BC'] = result_sdg_new.apply(find_intersection_ABC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."
...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,..."


In [33]:
# Apply the function to the merged dataframe
result_sdg_new['A'] = result_sdg_new.apply(find_difference_A, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[]
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[]
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[]
...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[]
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[]
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]"
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ..."


In [34]:
# Apply the function to the merged dataframe
result_sdg_new['B_and_C'] = result_sdg_new.apply(find_intersection_BC, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[],"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[],"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."
...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[],"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139..."


In [35]:
# Apply the function to the merged dataframe
result_sdg_new['B_minus_all'] = result_sdg_new.apply(find_difference_B, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[],"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[806, 815, 846, 847, 848, 892, 893, 894, 895, ..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[195, 199, 203, 207, 211, 215, 219, 222, 226, ..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[],"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[364, 1403, 1404, 1424, 379, 415, 437, 576, 58..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[582, 583, 1376, 1377, 1378, 1379, 1380, 1381,..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[132, 133, 140, 141, 143, 144, 147, 148, 149, ..."
...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[],"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[1122, 1123, 1124, 1125, 1126, 834, 835, 836, ..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[24, 25, 26, 32, 35, 36, 37, 47, 48, 50, 51, 5..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[1343, 1344, 1345, 1346, 1351, 1352, 1353, 135..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1459, 1473, 1474, 1444, 1445, 1447, 1448, 145..."


In [36]:
# Apply the function to the merged dataframe
result_sdg_new['C_minus_all'] = result_sdg_new.apply(find_difference_C, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[],"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[806, 815, 846, 847, 848, 892, 893, 894, 895, ...",[]
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[195, 199, 203, 207, 211, 215, 219, 222, 226, ...",[]
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[],"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[364, 1403, 1404, 1424, 379, 415, 437, 576, 58...",[]
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[582, 583, 1376, 1377, 1378, 1379, 1380, 1381,...",[]
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[132, 133, 140, 141, 143, 144, 147, 148, 149, ...",[]
...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[],"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[1122, 1123, 1124, 1125, 1126, 834, 835, 836, ...",[]
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[24, 25, 26, 32, 35, 36, 37, 47, 48, 50, 51, 5...",[]
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[1343, 1344, 1345, 1346, 1351, 1352, 1353, 135...",[]
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[163, 164, 165, 166, 167, 168, 169, 170, 171, ...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1459, 1473, 1474, 1444, 1445, 1447, 1448, 145...",[]


In [37]:
result_sdg_new.columns

Index(['Goal No.', 'Target_id', 'top_n_attr_A', 'top_n_attr_B', 'top_n_attr_C',
       'B_plus_C', 'A_and_BC', 'A', 'B_and_C', 'B_minus_all', 'C_minus_all'],
      dtype='object')

In [38]:
# Apply the function to the merged dataframe
result_sdg_new['Rank'] = result_sdg_new.apply(final_rank, axis=1)
result_sdg_new

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[],"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[806, 815, 846, 847, 848, 892, 893, 894, 895, ...",[],"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3..."
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[195, 199, 203, 207, 211, 215, 219, 222, 226, ...",[],"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107..."
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[],"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[364, 1403, 1404, 1424, 379, 415, 437, 576, 58...",[],"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,..."
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[582, 583, 1376, 1377, 1378, 1379, 1380, 1381,...",[],"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119..."
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[132, 133, 140, 141, 143, 144, 147, 148, 149, ...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155..."
...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[],"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[1122, 1123, 1124, 1125, 1126, 834, 835, 836, ...",[],"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,..."
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[24, 25, 26, 32, 35, 36, 37, 47, 48, 50, 51, 5...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,..."
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[1343, 1344, 1345, 1346, 1351, 1352, 1353, 135...",[],"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ..."
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1525, 1526, 1527, 15

In [39]:
final_df = result_sdg_new.copy()

In [40]:
final_df['Attr_count'] = final_df['Rank'].apply(lambda x: len(x))

In [41]:
final_df

,Goal No.,Target_id,top_n_attr_A,top_n_attr_B,top_n_attr_C,B_plus_C,A_and_BC,A,B_and_C,B_minus_all,C_minus_all,Rank,Attr_count
0,1,1.1,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",[],"[318, 319, 321, 824, 826, 1392, 1393, 1394, 13...","[806, 815, 846, 847, 848, 892, 893, 894, 895, ...",[],"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",557
1,1,1.2,"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...","[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[195, 199, 203, 207, 211, 215, 219, 222, 226, ...",[],"[1526, 1064, 1065, 1066, 1067, 1068, 1069, 107...",572
2,1,1.3,"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",[],"[389, 475, 1170, 1171, 1224, 1225, 1226, 1227,...","[364, 1403, 1404, 1424, 379, 415, 437, 576, 58...",[],"[389, 475, 1168, 1169, 1170, 1171, 1224, 1225,...",287
3,1,1.4,"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...","[23, 334, 335, 336, 337, 338, 339, 340, 341, 3...","[362, 363, 364, 1403, 1404, 1424, 584, 585, 58...","[582, 583, 1376, 1377, 1378, 1379, 1380, 1381,...",[],"[1170, 1171, 1193, 1194, 1195, 1196, 1197, 119...",598
4,1,1.5,"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...","[132, 133, 140, 141, 143, 144, 147, 148, 149, ...",[],"[1547, 1548, 1549, 1550, 1551, 1552, 1553, 155...",502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,16,16.3,"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",[],"[389, 475, 1170, 1171, 1011, 1012, 1013, 1014,...","[1122, 1123, 1124, 1125, 1126, 834, 835, 836, ...",[],"[389, 475, 1011, 1012, 1013, 1014, 1015, 1016,...",278
127,16,16.4,"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...","[24, 25, 26, 32, 35, 36, 37, 47, 48, 50, 51, 5...",[],"[580, 581, 582, 583, 584, 585, 586, 587, 1341,...",268
128,16,16.5,"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...","[540, 541, 542, 543, 544, 545, 824, 826]","[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, ...","[1343, 1344, 1345, 1346, 1351, 1352, 1353, 135...",[],"[1526, 1529, 1534, 1537, 1539, 1541, 1543, 3, ...",297
129,17,17.1,"[1525, 1526, 1527, 1531, 1532, 63, 64, 65, 66,...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1390, 1391, 1392, 139...","[1006, 1376, 1377, 1389, 1

In [42]:
final_df.to_csv(absolute_path_rank, sep=';', index=False)

In [43]:
all_goals = process_result(final_df, sdg, data)

In [44]:
all_goals.to_csv(absolute_path_rank_unroll, sep=';', index=False)

In [45]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
41108,1537,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Bank Post Office Account Holders,Sandhya Suraksha","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41109,1538,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Pensioners,Mythri Pension Scheme","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41110,1539,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Bank Post Office Account Holders,Mythri Pensio...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41111,1540,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Pensioners,Manaswini Pension Scheme","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [46]:
# Group by 'col1' and count the distinct values in 'col2'
chap_map = data_new.groupby('Chapter_id')['Attr_id'].nunique().to_dict()
chap_map

{'1': 67,
 '10': 128,
 '11': 73,
 '12': 117,
 '13': 35,
 '14': 33,
 '15': 69,
 '16': 43,
 '17': 42,
 '2': 241,
 '3': 16,
 '4': 456,
 '5': 44,
 '6': 83,
 '7': 32,
 '8': 28,
 '9': 46}

In [47]:
# Group by 'col1' and count the distinct values in 'col2'
table_map = data_new.groupby('Chapter_id')['Table_id'].nunique().to_dict()
table_map

{'1': 10,
 '10': 15,
 '11': 6,
 '12': 6,
 '13': 6,
 '14': 5,
 '15': 9,
 '16': 3,
 '17': 3,
 '2': 21,
 '3': 2,
 '4': 19,
 '5': 4,
 '6': 4,
 '7': 2,
 '8': 4,
 '9': 4}

In [48]:
attr_map = data_new.groupby('Table_id')['Attr_id'].nunique().to_dict()
attr_map

{'1.1': 4,
 '1.10': 7,
 '1.2': 3,
 '1.3': 6,
 '1.4': 4,
 '1.5': 4,
 '1.6': 15,
 '1.7': 8,
 '1.8': 8,
 '1.9': 8,
 '10.1': 9,
 '10.10': 10,
 '10.11': 11,
 '10.12': 9,
 '10.13': 24,
 '10.14': 8,
 '10.15': 8,
 '10.2': 10,
 '10.3': 4,
 '10.4': 4,
 '10.5': 4,
 '10.6': 5,
 '10.7': 5,
 '10.8': 7,
 '10.9': 10,
 '11.1': 14,
 '11.2': 16,
 '11.3': 7,
 '11.4': 5,
 '11.5': 23,
 '11.6': 8,
 '12.1': 24,
 '12.2': 18,
 '12.3': 30,
 '12.4': 6,
 '12.5': 18,
 '12.6': 21,
 '13.1': 6,
 '13.2': 4,
 '13.3': 5,
 '13.4': 5,
 '13.5': 8,
 '13.6': 7,
 '14.1': 7,
 '14.2': 6,
 '14.3': 7,
 '14.4': 7,
 '14.5': 6,
 '15.1': 7,
 '15.2': 6,
 '15.3': 9,
 '15.4': 8,
 '15.5': 5,
 '15.6': 9,
 '15.7': 10,
 '15.8': 10,
 '15.9': 5,
 '16.1': 7,
 '16.2': 16,
 '16.3': 20,
 '17.1': 4,
 '17.2': 19,
 '17.3': 19,
 '2.1': 18,
 '2.11': 20,
 '2.12': 11,
 '2.13': 34,
 '2.14': 10,
 '2.15': 10,
 '2.16': 10,
 '2.17': 10,
 '2.18': 10,
 '2.19': 10,
 '2.2': 10,
 '2.20': 10,
 '2.21': 10,
 '2.22': 10,
 '2.3': 7,
 '2.4': 12,
 '2.5': 7,
 '2.6': 10,
 

In [49]:
all_goals

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
41108,1537,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Bank Post Office Account Holders,Sandhya Suraksha","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41109,1538,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Pensioners,Mythri Pension Scheme","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41110,1539,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Bank Post Office Account Holders,Mythri Pensio...","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41111,1540,17,Other Information,17.2,Number of Pensioners under Social Security Sch...,"Pensioners,Manaswini Pension Scheme","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [50]:
filter_df = all_goals.copy()

In [51]:
# Initialize an empty dictionary to store the counts
tab_result = {}
table_id_rm = {}
# Iterate over unique values in col1
for val in filter_df['top_n_target_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = filter_df[filter_df['top_n_target_id'] == val]
    
    # Count the occurrences of each value in col2 and store in a dictionary
    table_counts = filtered_df['Table_id'].value_counts().to_dict()
    
    table_id_rm[val] = {key:value for key, value in table_counts.items() if value < (attr_map[key]/2)}
    
    table_dict_new = {key:value for key, value in table_counts.items() if key not in table_id_rm[val]}
    
    if(len(table_dict_new) <= 4):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(table_counts.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_6_dict = dict(list(sorted_dict.items())[:6])
        tab_result[val] = top_6_dict
        
    else:
        # Add the col2 counts dictionary to the result dictionary
        tab_result[val] = table_dict_new

print(tab_result)

{'1.1': {'4.19': 197, '2.13': 34, '2.11': 20, '2.1': 18, '1.6': 15, '2.4': 12, '2.12': 11, '2.15': 10, '2.6': 10, '2.16': 10, '2.8': 10, '2.17': 10, '2.21': 10, '2.18': 10, '2.19': 10, '2.2': 10, '2.20': 10, '2.22': 10, '2.14': 10, '1.8': 8, '1.9': 8, '1.7': 8, '2.5': 7, '2.3': 7, '11.3': 7, '2.9': 6, '2.7': 6, '3.1': 5, '6.3': 4, '10.4': 4, '14.3': 4}, '1.2': {'4.19': 197, '2.11': 20, '4.2': 20, '2.13': 18, '17.2': 15, '1.6': 15, '11.5': 14, '6.1': 14, '2.4': 12, '15.3': 9, '1.7': 8, '1.8': 8, '13.5': 8, '1.9': 8, '10.8': 7, '2.21': 7, '2.5': 7, '13.6': 7, '14.3': 7, '11.3': 7, '13.1': 6, '2.16': 6, '10.1': 6, '2.22': 6, '2.20': 6, '15.2': 6, '2.19': 6, '2.18': 6, '10.2': 6, '2.17': 6, '2.15': 6, '2.14': 6, '13.4': 5, '13.3': 5, '13.2': 4, '1.10': 4, '1.4': 2}, '1.3': {'12.3': 30, '12.1': 24, '12.6': 21, '17.2': 19, '12.2': 18, '12.5': 18, '10.13': 12, '14.3': 7, '12.4': 6, '13.2': 4, '10.15': 4, '10.14': 4, '11.4': 3}, '1.4': {'4.19': 197, '12.3': 30, '12.1': 24, '11.5': 23, '12.6': 

In [52]:
len(tab_result)

131

In [53]:
table_id_rm

{'1.1': {'17.2': 7,
  '4.18': 6,
  '6.2': 4,
  '5.4': 4,
  '15.3': 4,
  '6.1': 3,
  '4.2': 3,
  '15.8': 2,
  '16.3': 2,
  '16.2': 2,
  '15.7': 2,
  '8.4': 2,
  '11.6': 2,
  '14.5': 2,
  '4.4': 2,
  '15.9': 1,
  '15.5': 1,
  '15.4': 1,
  '5.2': 1,
  '5.1': 1,
  '1.5': 1,
  '14.1': 1,
  '4.3': 1,
  '9.1': 1},
 '1.2': {'6.2': 26,
  '17.3': 8,
  '11.1': 4,
  '2.1': 4,
  '2.2': 4,
  '2.6': 4,
  '2.8': 4,
  '4.18': 3,
  '14.1': 3,
  '11.6': 3,
  '10.7': 2,
  '10.11': 2,
  '10.6': 2,
  '9.2': 1,
  '1.5': 1,
  '15.1': 1},
 '1.3': {'4.19': 49,
  '11.5': 6,
  '4.18': 6,
  '11.2': 4,
  '10.12': 4,
  '6.2': 4,
  '11.6': 3,
  '4.2': 3,
  '6.1': 3,
  '2.13': 3,
  '16.3': 2,
  '16.2': 2,
  '15.8': 2,
  '15.7': 2,
  '8.4': 2,
  '4.3': 2,
  '4.4': 2,
  '15.3': 2,
  '14.5': 2,
  '10.11': 2,
  '4.15': 2,
  '5.2': 1,
  '5.1': 1,
  '9.1': 1,
  '15.4': 1,
  '15.5': 1,
  '15.9': 1,
  '4.7': 1,
  '6.3': 1,
  '11.3': 1,
  '1.5': 1},
 '1.4': {'6.2': 12,
  '4.18': 10,
  '6.1': 7,
  '9.1': 7,
  '7.2': 3,
  '4.2':

In [54]:
sum_ind=0
# Iterate over the outer dictionary
for key, inner_dict in table_id_rm.items():
    # Iterate over the inner dictionary
    for k, value in inner_dict.items():
        if k not in tab_result[key]:
            # Add the value to the result
            sum_ind += value

print(sum_ind)

6449


In [55]:
# List to store indices to remove
indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in filter_df.iterrows():
    if row['top_n_target_id'] in table_id_rm:
        inner_dict = table_id_rm[row['top_n_target_id']]
        if row['Table_id'] in inner_dict:
            if row['Table_id'] not in tab_result[row['top_n_target_id']]:
                # Add index to remove list
                indices_to_remove.append(index)
            
indices_to_remove

[241,
 242,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 273,
 278,
 279,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 310,
 311,
 555,
 556,
 747,
 755,
 756,
 757,
 907,
 908,
 909,
 911,
 912,
 913,
 914,
 915,
 916,
 917,
 918,
 972,
 973,
 974,
 975,
 976,
 977,
 978,
 979,
 980,
 981,
 982,
 983,
 984,
 991,
 992,
 993,
 1001,
 1002,
 1003,
 1004,
 1006,
 1007,
 1008,
 1009,
 1010,
 1011,
 1012,
 1013,
 1057,
 1058,
 1059,
 1060,
 1061,
 1062,
 1063,
 1064,
 1065,
 1066,
 1067,
 1068,
 1069,
 1076,
 1077,
 1078,
 1086,
 1087,
 1088,
 1089,
 1115,
 1116,
 1117,
 1118,
 1119,
 1120,
 1121,
 1122,
 1123,
 1129,
 1130,
 1131,
 1132,
 1133,
 1134,
 1271,
 1272,
 1273,
 1274,
 1275,
 1276,
 1284,
 1285,
 1286,
 1287,
 1288,
 1292,
 1293,
 1294,
 1295,
 1296,
 1297,
 1298,
 1299,
 1300,
 1301,
 1

In [56]:
len(indices_to_remove)

6449

In [57]:
# Remove rows using indices from indices_to_remove list
filter_df.drop(indices_to_remove, inplace=True)

filter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
0,24,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
1,25,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
2,26,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Total,Ration shops",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
3,27,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Urban,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
4,28,1,General Information,1.6,Priority Non Priority and Total Ration Card Ho...,"Rural,Anthyodaya,Ration card holders",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
41102,1086,10,Education,10.10,Infrastructure Facilities in High Schools on 3...,Ramp,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41103,1087,10,Education,10.10,Infrastructure Facilities in High Schools on 3...,Compound,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41104,1088,10,Education,10.10,Infrastructure Facilities in High Schools on 3...,Drinking Water,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41105,1089,10,Education,10.10,Infrastructure Facilities in High Schools on 3...,Library,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [58]:
filter_df.to_csv(absolute_path_tab_filter, sep=';', index=False)

In [59]:
tab_count_att = filter_df['top_n_target_id'].value_counts().to_dict()

In [90]:
filter_df_copy = filter_df.copy()

In [91]:
def filter_dataframe(df, n, chap_list):
    result_df = df.copy() # Initialize an empty DataFrame to store the filtered rows
    
    # Loop over each unique 'Target_id'
    for target_id, group in df.groupby('top_n_target_id'):
        num_rows = len(group)  # Count the number of rows for this 'Target_id'
        
        # Check if the number of rows is greater than the threshold 'n'
        if num_rows > n:
            idx_list = []
            drop_list = []
            for chap in chap_list:
                # Find the indices corresponding to chapters "2" and "3" for this 'Target_id'
                idx_chapter = group[group['Chapter_id'] == chap].index
                idx_list.append(idx_chapter)
                
            for item in idx_list:
                if len(item)>1:
                    if len(item)>350:
                        total_rows_to_drop = item[len(item)//3:]
                    else:
                        total_rows_to_drop = item[len(item)//2:]
                    drop_list.append(total_rows_to_drop)
            
            for i in drop_list:
                if len(i)>0:
                    result_df = result_df.drop(i)
    result_df.reset_index(inplace = True, drop = True)
    return result_df

In [92]:
chap_ll = ['2', '4', '10', '12']
# Filter the DataFrame using the function
final_map = filter_dataframe(filter_df_copy, 350, chap_ll)

In [63]:
final_map.to_csv(absolute_path_final_map, sep=';', index=False)

In [64]:
chapter_df = filter_df.copy()

In [65]:
# Initialize an empty dictionary to store the counts
chap_result = {}
chap_id_rm = {}

# Iterate over unique values in col1
for val in chapter_df['top_n_target_id'].unique():
    # Filter the dataframe by the current value of col1
    filtered_df = chapter_df[chapter_df['top_n_target_id'] == val]
    chap_list = []
    for tab in filtered_df['Table_id'].unique():
        chap_filter = filtered_df[filtered_df['Table_id'] == tab]
        
        # Count the occurrences of each value in col2 and store in a dictionary
        chap_counts = chap_filter['Chapter_id'].unique().tolist()
        
        chap_list.extend(chap_counts)
    chap_dict = pd.Series(chap_list).value_counts().to_dict()
    chap_id_rm[val] = {key:value for key, value in chap_dict.items() if value < (table_map[key]/2)}
    chap_dict_new = {key:value for key, value in chap_dict.items() if key not in chap_id_rm[val]}
    if(len(chap_dict_new) <= 1):
        # Sort the dictionary based on values in descending order
        sorted_dict = dict(sorted(chap_dict.items(), key=lambda x: x[1], reverse=True))
        # Keep only the top 2 elements
        top_4_dict = dict(list(sorted_dict.items())[:4])
        chap_result[val] = top_4_dict
 
    else:
        # Add the col2 counts dictionary to the result dictionary
        chap_result[val] = chap_dict_new

print(chap_result)

{'1.1': {'2': 21, '3': 1}, '1.2': {'2': 13, '13': 6, '1': 6}, '1.3': {'12': 6, '10': 3, '17': 1, '14': 1}, '1.4': {'11': 6, '13': 6, '12': 6, '14': 5}, '1.5': {'1': 10, '17': 3}, '1.6': {'13': 6, '14': 5}, '2.1': {'14': 3, '11': 3}, '2.2': {'13': 6, '1': 4, '2': 3, '14': 2}, '2.3': {'1': 10, '11': 6, '8': 4, '17': 3}, '2.4': {'4': 8, '2': 6, '8': 4, '13': 2}, '2.5': {'1': 10, '13': 6, '14': 5, '5': 4, '17': 3, '8': 3, '7': 2}, '2.6': {'4': 11, '11': 6, '13': 6, '14': 5, '8': 4, '7': 2, '5': 2}, '2.7': {'13': 6, '14': 5, '6': 4, '8': 4}, '2.8': {'1': 10, '17': 3}, '3.1': {'13': 6, '17': 2}, '3.2': {'13': 6, '10': 4, '2': 3, '11': 2}, '3.3': {'11': 6, '14': 3, '3': 2}, '3.4': {'11': 6, '13': 6}, '3.5': {'9': 4, '11': 4}, '3.6': {'10': 15, '1': 10, '11': 6, '17': 3}, '3.7': {'11': 6, '4': 6, '9': 1, '14': 1}, '3.8': {'10': 4, '14': 2, '17': 2, '4': 1}, '3.9': {'11': 6, '13': 6, '14': 5}, '3.10': {'1': 10, '11': 6, '13': 6, '14': 5, '17': 3, '12': 3}, '3.11': {'11': 6, '13': 6, '14': 5}, '

In [66]:
chap_id_rm

{'1.1': {'1': 4, '4': 1, '14': 1, '11': 1, '6': 1, '10': 1},
 '1.2': {'10': 3, '4': 2, '11': 2, '15': 2, '17': 1, '14': 1, '6': 1},
 '1.3': {'10': 3, '17': 1, '14': 1, '11': 1, '13': 1},
 '1.4': {'1': 4, '4': 2, '6': 1, '9': 1, '17': 1},
 '1.5': {'4': 8, '2': 3, '13': 2, '11': 1, '16': 1, '8': 1},
 '1.6': {'1': 4, '10': 4, '2': 3, '4': 1, '17': 1, '12': 1},
 '2.1': {'9': 1, '4': 1, '1': 1},
 '2.2': {'1': 4, '2': 3, '14': 2, '10': 1, '11': 1, '4': 1},
 '2.3': {'2': 4, '14': 2, '6': 1, '4': 1},
 '2.4': {'4': 8,
  '2': 6,
  '13': 2,
  '5': 1,
  '9': 1,
  '10': 1,
  '11': 1,
  '16': 1,
  '17': 1},
 '2.5': {'4': 9, '11': 2, '9': 1, '2': 1, '10': 1, '15': 1},
 '2.6': {'2': 4, '10': 2, '1': 1, '9': 1},
 '2.7': {'2': 4, '4': 3, '11': 1},
 '2.8': {'13': 2, '9': 1, '14': 1, '11': 1},
 '3.1': {'10': 4, '2': 3, '11': 2, '1': 1, '16': 1},
 '3.2': {'10': 4, '2': 3, '11': 2, '17': 1, '14': 1, '12': 1},
 '3.3': {'1': 4, '9': 1, '15': 1, '4': 1},
 '3.4': {'2': 3, '14': 2, '10': 2, '15': 1},
 '3.5': {'1

In [67]:
missing = []

# Iterate through the data dictionary
for key, value in chap_result.items():
    # Check if the sub-dictionary is empty
    if not value:
        # Add the key to the missing list
        missing.append(key)
missing

[]

In [68]:
# List to store indices to remove
chap_indices_to_remove = []
# Iterate over the rows of the DataFrame
for index, row in chapter_df.iterrows():
    if row['top_n_target_id'] in table_id_rm:
        inner_dict = chap_id_rm[row['top_n_target_id']]
        if row['Chapter_id'] in inner_dict:
            if row['top_n_target_id'] not in chap_result or row['Chapter_id'] not in chap_result[row['top_n_target_id']]:
                # Add index to remove list
                chap_indices_to_remove.append(index)

chap_indices_to_remove

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 

In [69]:
len(chap_indices_to_remove)

13242

In [70]:
# Remove rows using indices from indices_to_remove list
chapter_df.drop(chap_indices_to_remove, inplace=True)

chapter_df

,Attr_id,Chapter_id,Chapter_name,Table_id,Table_name,Description,Goal,Targets,top_n_goal_id,top_n_target_id
39,318,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Working,Raingauge Stations",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
40,319,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Under Repair,Raingauge Stations",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
41,320,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Total,Raingauge Stations",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
42,321,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Rainy Days,Annual Normal Rainfall 1961-2010",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
43,322,3,Rainfall,3.1,No. of Raingauge Stations Annual Normal Rainfa...,"Normal Rainfall,Annual Normal Rainfall 1961-2010",End poverty in all its forms everywhere,"By 2030, eradicate extreme poverty for all peo...",1,1.1
...,...,...,...,...,...,...,...,...,...,...
41085,1492,16,Energy,16.2,Sectorwise number of Electricity Consumers as ...,"Lt,I.p. Sets","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41086,1493,16,Energy,16.2,Sectorwise number of Electricity Consumers as ...,Street Light Lt,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41087,1494,16,Energy,16.2,Sectorwise number of Electricity Consumers as ...,Water supply Ht,"Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2
41088,1495,16,Energy,16.2,Sectorwise number of Electricity Consumers as ...,"Lt,Water Supply","Data, monitoring and accountability","By 2030, build on existing initiatives to deve...",17,17.2


In [71]:
chapter_df['top_n_target_id'].nunique()

131

In [72]:
chap_count_att = chapter_df['top_n_target_id'].value_counts().to_dict()

In [73]:
chap_count_att

{'6.6': 573,
 '2.6': 536,
 '11.8': 462,
 '5.6': 418,
 '2.4': 368,
 '8.10': 353,
 '15.7': 350,
 '10.1': 329,
 '3.6': 310,
 '4.8': 301,
 '4.2': 298,
 '4.3': 292,
 '14.8': 292,
 '3.10': 286,
 '4.9': 283,
 '14.3': 280,
 '2.5': 275,
 '8.6': 274,
 '16.5': 267,
 '12.5': 260,
 '1.4': 258,
 '11.1': 251,
 '9.3': 246,
 '1.1': 246,
 '10.4': 245,
 '5.9': 244,
 '4.4': 237,
 '9.4': 233,
 '5.8': 223,
 '14.6': 220,
 '16.4': 213,
 '2.3': 210,
 '4.1': 208,
 '8.3': 199,
 '9.2': 196,
 '9.6': 196,
 '4.7': 196,
 '7.3': 194,
 '1.2': 192,
 '5.4': 190,
 '8.9': 187,
 '16.2': 187,
 '10.2': 185,
 '13.1': 182,
 '2.7': 179,
 '12.7': 177,
 '5.7': 177,
 '13.3': 172,
 '11.4': 168,
 '4.5': 163,
 '1.3': 163,
 '4.6': 162,
 '3.7': 160,
 '9.7': 157,
 '8.7': 157,
 '9.8': 155,
 '5.5': 152,
 '16.3': 151,
 '12.9': 151,
 '14.5': 150,
 '12.4': 146,
 '15.3': 145,
 '6.2': 144,
 '14.12': 144,
 '3.11': 141,
 '3.9': 141,
 '6.7': 141,
 '14.1': 141,
 '14.4': 141,
 '9.5': 140,
 '12.3': 136,
 '6.1': 136,
 '6.8': 134,
 '7.5': 133,
 '13.2':

In [74]:
tab_count_att

{'11.8': 840,
 '2.6': 611,
 '6.6': 582,
 '1.4': 530,
 '2.3': 515,
 '14.1': 505,
 '1.1': 501,
 '1.2': 500,
 '2.5': 482,
 '1.5': 476,
 '5.6': 476,
 '4.3': 464,
 '5.7': 461,
 '2.7': 455,
 '9.2': 442,
 '14.2': 436,
 '2.4': 431,
 '11.9': 430,
 '15.7': 421,
 '15.1': 397,
 '10.1': 391,
 '4.6': 390,
 '15.5': 387,
 '1.6': 385,
 '8.3': 378,
 '4.4': 371,
 '4.8': 370,
 '8.10': 364,
 '4.1': 363,
 '14.8': 361,
 '3.3': 361,
 '9.5': 351,
 '3.10': 343,
 '2.2': 338,
 '14.9': 338,
 '3.6': 336,
 '4.9': 335,
 '15.2': 331,
 '9.4': 326,
 '7.1': 323,
 '12.5': 322,
 '8.6': 322,
 '10.4': 316,
 '4.2': 305,
 '14.3': 303,
 '5.4': 303,
 '14.4': 296,
 '5.9': 284,
 '10.2': 279,
 '10.3': 279,
 '4.7': 277,
 '16.5': 275,
 '11.1': 274,
 '5.5': 272,
 '7.3': 270,
 '8.4': 266,
 '9.3': 265,
 '16.2': 260,
 '8.9': 257,
 '3.9': 253,
 '13.1': 251,
 '14.6': 249,
 '11.4': 248,
 '12.6': 246,
 '16.4': 246,
 '9.7': 243,
 '8.7': 243,
 '5.8': 231,
 '13.3': 230,
 '12.8': 229,
 '15.3': 226,
 '12.3': 219,
 '7.2': 217,
 '15.9': 215,
 '4.5'

In [75]:
rank_count_att = all_goals['top_n_target_id'].value_counts().to_dict()

In [93]:
rank_filter_count_att = final_map['top_n_target_id'].value_counts().to_dict()

In [77]:
count_att_list = [(key, value) for key, value in chap_count_att.items()]

count_att_df = pd.DataFrame(count_att_list, columns=['Target_id', 'Chap_count'])
count_att_df

,Target_id,Chap_count
0,6.6,573
1,2.6,536
2,11.8,462
3,5.6,418
4,2.4,368
...,...,...
126,5.3,49
127,6.3,47
128,5.1,40
129,2.1,38


In [78]:
count_att_df['Tab_count'] = count_att_df['Target_id'].map(tab_count_att)

In [94]:
count_att_df['Filter_count'] = count_att_df['Target_id'].map(rank_filter_count_att)

In [80]:
count_att_df['Rank_count'] = count_att_df['Target_id'].map(rank_count_att)

In [95]:
count_att_df

,Target_id,Chap_count,Tab_count,Filter_count,Rank_count
0,6.6,573,582,306,621
1,2.6,536,611,426,716
2,11.8,462,840,494,861
3,5.6,418,476,327,517
4,2.4,368,431,263,457
...,...,...,...,...,...
126,5.3,49,151,151,304
127,6.3,47,63,63,85
128,5.1,40,100,100,309
129,2.1,38,73,73,185


In [82]:
chapter_df.to_csv(absolute_path_chap_filter, sep=';', index=False)

In [83]:
count_att_df.to_csv(absolute_path_count, sep=';', index=False)